In [24]:
# Make necessary imports
from dotenv import load_dotenv
from openai import OpenAI
from agents import Agent, trace, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Email, To, Content
import os
import asyncio


In [25]:
load_dotenv(override=True)

True

In [20]:
# test sendgrid api by sending a mail
def send_test_email():
    sg_client=SendGridAPIClient(api_key=os.getenv(key="SENDGRID_API_KEY"))
    from_email = Email("sumanprakash.nitp@gmail.com")
    to_email = To("bittu1suman2@gmail.com")
    content= Content("text/plain","This is a test email.")
    mail = Mail(from_email,to_email,"Test Email",content).get()
    response = sg_client.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


In [22]:
def send_email():
    message = Mail(
    from_email='sumanprakash.nitp@gmail.com',
    to_emails='bittu1suman2@gmail.com',
    subject='Sending with Twilio SendGrid is Fun',
    html_content='<strong>and easy to do anywhere, even with Python</strong>')
    try:
        sg = SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
    # sg.set_sendgrid_data_residency("eu")
    # uncomment the above line if you are sending mail using a regional EU subuser
        response = sg.send(message)
        print(response.status_code)
        print(response.body)
        print(response.headers)
    except Exception as e:
        print(e.message)

send_email()

202
b''
Server: nginx
Date: Thu, 05 Feb 2026 15:33:45 GMT
Content-Length: 0
Connection: close
X-Message-Id: GTB_zP5MQGiLzHbPZ1KMaw
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin




In [23]:
# Create 3 System Instructions for 3 different sales agent
instruction1 = f""" You are a ptofessional sales agent working for OOIMA FAASHION, a company that deals in apparel business
especially custom products like printed t-shirts, caps, mouse pads, mugs. Your job is to write professional and serious
cold emails. 
"""
instruction2 = f""" You are a humorous, engaging sales agent working for OOIMA FAASHION, a company that deals in apparel business
especially custom products like printed t-shirts, caps, mouse pads, mugs. Your job is to write witty, engaging
cold emails that are likely to get a response. 
"""
instruction3 = f""" You are a busy sales agent working for OOIMA FAASHION, a company that deals in apparel business
especially custom products like printed t-shirts, caps, mouse pads, mugs. Your job is to concise, to the point cold emails. 
"""

In [26]:
#Now create 3 different sales agent
sales_agent1= Agent(name="Professional sales agent", instructions=instruction1, model="gpt-4o-mini")
sales_agent2= Agent(name="Humorous sales agent", instructions=instruction2, model="gpt-4o-mini")
sales_agent3= Agent(name="Busy sales agent", instructions=instruction3, model="gpt-4o-mini")


In [27]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Elevate Your Brand with Custom Apparel from OOIMA FASHION

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent OOIMA FASHION, a leader in custom apparel solutions tailored specifically for brands like yours.

At OOIMA FASHION, we specialize in high-quality printed products, including t-shirts, caps, mouse pads, and mugs. Our focus on customization allows you to create unique merchandise that resonates with your audience and enhances your brand identity.

Why choose OOIMA FASHION?

- **Quality**: Our products are made from premium materials and printed with advanced techniques to ensure durability and vibrant designs.
- **Flexibility**: Whether you need small batches for an upcoming event or large orders for a promotional campaign, we can accommodate your needs.
- **Customer-Centric Service**: Our dedicated team works closely with you from concept to delivery, ensuring a seamless experience every step of the way.

I would love the

In [28]:
sales_agents = [sales_agent1, sales_agent2, sales_agent3]

In [34]:
# Run all 3 agents 
message = "write a cold sales mail"
with trace("Parallel Cold Email"):
    results = await asyncio.gather( 
        Runner.run(sales_agent1,message),
        Runner.run(sales_agent2,message),
        Runner.run(sales_agent3,message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output+"\n\n")


Subject: Elevate Your Brand with Custom Apparel from OOIMA FAASHION

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent OOIMA FAASHION, a leading provider of custom apparel and promotional products.

In today's competitive market, effectively showcasing your brand is essential. Our high-quality products—ranging from printed t-shirts and caps to custom mugs and mouse pads—are designed to enhance your brand’s visibility and appeal to your target audience.

Here are a few benefits of partnering with us:

1. **Customization:** Tailor products to reflect your brand’s unique identity. 
2. **Quality:** We use premium materials ensuring durability and comfort.
3. **Scalability:** Whether you need a single item or a bulk order, we can accommodate your needs.

I would love the opportunity to discuss how OOIMA FAASHION can help elevate your brand. Are you available for a brief call next week?

Thank you for considering us. I look forward to your 

In [37]:
# Create a new Agent Sales picker whose job is to select and pick the best mail out of the ones written by sales agents
sales_picker = Agent(name="Sales_Picker", 
instructions=f"""You pick the best cold sale emails form the given options. Imagine yourself as a customer 
and pick the one you are highly likely to respond to. Do not give any explanations. 
Just reply with the best cold sale email from the give options.""",
model="gpt-4o-mini")

In [40]:
# Use sales agent to write cold sales email
message = "Please write a cold sale email"
results = await asyncio.gather(
    Runner.run(sales_agent1,message),
    Runner.run(sales_agent2,message),
    Runner.run(sales_agent3,message)
)
outputs = [result.final_output for result in results]
#print(outputs)

# Consolidate the generated cold sale emails
emails = f"Cold Sales Emails:\n\nEmail:\n\n".join(outputs) 
print(emails)
best_email = await Runner.run(sales_picker,emails)
print(f"Best cold sale Email:\n\n {best_email.final_output}")

Subject: Elevate Your Brand with Custom Apparel and Merchandise

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am a representative of OOIMA FASHION. We specialize in high-quality custom apparel and merchandise, including printed t-shirts, caps, mouse pads, and mugs, tailored to help brands like yours stand out.

In today’s competitive marketplace, unique and personalized merchandise can significantly enhance your brand’s visibility and customer engagement. At OOIMA FASHION, we take pride in offering:

- **Exceptional Quality**: Our products are made with premium materials to ensure durability and comfort.
- **Customization Options**: We provide a wide range of designs, colors, and sizes to reflect your brand identity perfectly.
- **Fast Turnaround**: We understand the value of time; hence, we ensure prompt production and delivery without compromising quality.

I would love to explore how our products can support your marketing efforts and d